# Get encoded atoms

In [1]:
import pandas as pd
import os

import sys
sys.path.append('../../../../icml18-jtnn')
sys.path.append('../../../../icml18-jtnn/jtnn')

In [2]:
from tqdm import tqdm

In [3]:
VOCAB_PATH = '../../../../icml18-jtnn/data/zinc/vocab.txt'
MODEL_PATH = '../../../../icml18-jtnn/molvae'
DATASET_PATH = '../../../data/3_final_data/split_data'

RAW_PATH = '../../../data/raw/baselines/jtree'

SMILES_COLUMN = 'smiles'
VALUE_COLUMN = 'logP'

In [4]:
import torch
import torch.nn as nn
from torch.autograd import Variable

import numpy as np

import math, random, sys
from optparse import OptionParser
from collections import deque

import rdkit
import rdkit.Chem as Chem

from jtnn import *

In [5]:
vocab = [x.strip("\r\n ") for x in open(VOCAB_PATH)] 
vocab = Vocab(vocab)

In [6]:
hidden_size = int(450)
latent_size = int(56)
depth = int(3)
stereo = True if int(1) == 1 else False

In [7]:
model = JTNNVAE(vocab, hidden_size, latent_size, depth, stereo=stereo)
model.load_state_dict(torch.load(os.path.join(MODEL_PATH, 'MPNVAE-h450-L56-d3-beta0.001/model.iter-4')))
model = model.cuda()

/home/mol/anaconda3/envs/jtree/lib/python2.7/site-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [8]:
dataset_train = pd.read_csv(os.path.join(DATASET_PATH, 'logp_wo_averaging_train.csv'), index_col=0)
dataset_val = pd.read_csv(os.path.join(DATASET_PATH, 'logp_wo_averaging_validation.csv'), index_col=0)
dataset_test = pd.read_csv(os.path.join(DATASET_PATH, 'logp_wo_averaging_test.csv'), index_col=0)

In [9]:
def create_latent_representation_dataset(df, SMILES_COLUMN, VALUE_COLUMN):
    import numpy as np    
    vectors = []
    broken_smiles = {}
    for smiles in tqdm(df[SMILES_COLUMN].values):
        try:
            latent_representation = model.encode_latent_mean([smiles])
        except (KeyError, RuntimeError), e:
            broken_smiles[smiles]=e
            continue
        vectors.append(latent_representation.cpu().detach().numpy())
    X = np.array(vectors)
    y = df[~df[SMILES_COLUMN].isin(list(broken_smiles.keys()))][VALUE_COLUMN].values
    assert len(X)==len(y)
    return X, y, broken_smiles

In [ ]:
X_train, y_train, train_errs = create_latent_representation_dataset(dataset_train, SMILES_COLUMN, VALUE_COLUMN)
with open(os.path.join(RAW_PATH,'X_train.npy'), 'wb') as f:
    np.save(f,X_train)
with open(os.path.join(RAW_PATH,'y_train.npy'), 'wb') as f:
    np.save(f,y_train)   
X_val, y_val, val_errs = create_latent_representation_dataset(dataset_val, SMILES_COLUMN, VALUE_COLUMN)
with open(os.path.join(RAW_PATH,'X_val.npy'), 'wb') as f:
    np.save(f,X_val)
with open(os.path.join(RAW_PATH,'y_val.npy'), 'wb') as f:
    np.save(f,y_val)   
X_test, y_test, test_errs = create_latent_representation_dataset(dataset_test, SMILES_COLUMN, VALUE_COLUMN)
with open(os.path.join(RAW_PATH,'X_test.npy'), 'wb') as f:
    np.save(f,X_test)
with open(os.path.join(RAW_PATH,'y_test.npy'), 'wb') as f:
    np.save(f,y_test)   

In [ ]:
with open(os.path.join(RAW_PATH,'train_errs.txt'), 'w') as f:
    f.write('\n'.join(list(train_errs.keys())))
with open(os.path.join(RAW_PATH,'val_errs.txt'), 'w') as f:
    f.write('\n'.join(list(val_errs.keys())))
with open(os.path.join(RAW_PATH,'test_errs.json'), 'w') as f:
    f.write('\n'.join(list(test_errs.keys())))

# Train model based on pretrained vectors 

In [18]:
fname = os.path.join(RAW_PATH, 'logs', 'exp_0')
os.makedirs(fname)
writer = SummaryWriter(fname)

In [14]:
from tensorboardX import SummaryWriter

## RidgeRegression

In [15]:
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [16]:
with open(os.path.join(RAW_PATH,'X_train.npy'), 'rb') as f:
    X_train = np.load(f).squeeze()
with open(os.path.join(RAW_PATH,'y_train.npy'), 'rb') as f:
    y_train = np.load(f)
    
y_train_transformed = scaler.fit_transform(y_train.reshape(-1, 1)).reshape(-1)
    
with open(os.path.join(RAW_PATH,'X_val.npy'), 'rb') as f:
    X_val = np.load(f).squeeze()
with open(os.path.join(RAW_PATH,'y_val.npy'), 'rb') as f:
    y_val = np.load(f)

with open(os.path.join(RAW_PATH,'X_test.npy'), 'rb') as f:
    X_test = np.load(f).squeeze()
with open(os.path.join(RAW_PATH,'y_test.npy'), 'rb') as f:
    y_test = np.load(f) 

### Train with whole vector

In [69]:
model = Ridge(alpha=4.5)

In [70]:
y_train_transformed = scaler.fit_transform(y_train.reshape(-1, 1)).reshape(-1)

In [71]:
model.fit(X_train, y_train_transformed)

Ridge(alpha=4.5, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [72]:
test_preds = scaler.inverse_transform(model.predict(X_test))
val_preds = scaler.inverse_transform(model.predict(X_val))

In [73]:
print("Valid RMSE =", mean_squared_error(y_val, val_preds)**0.5)
print("Valid R2-score is {0}".format(r2_score(y_val, val_preds)))

print("Test RMSE =", mean_squared_error(y_test, test_preds)**0.5)
print("Test R2-score is {0}".format(r2_score(y_test, test_preds)))

('Valid RMSE =', 1.2295349916179965)
Valid R2-score is 0.545469175613
('Test RMSE =', 1.258996302029825)
Test R2-score is 0.524584524731


In [74]:
feature_importance = model.coef_
features = list(range(X_train.shape[1]))
df = pd.DataFrame(columns = ['feature importance', 'feature name'])
df['feature importance'] = feature_importance
df['feature name'] = [str(i)+ '_tree_feats' for i in range(X_train.shape[1]//2)] +\
                    [str(i)+ '_graph_feats' for i in range(X_train.shape[1]//2, X_train.shape[1])]
df_20_most_important = df.sort_values(by='feature importance', ascending = False)[:20]

In [75]:
import pandas as pd
import os

In [76]:
for i in range(len(df_20_most_important)):
    print '|', df_20_most_important.iloc[i]['feature name'], '|', df_20_most_important.iloc[i]['feature importance'], '|'

| 42_graph_feats | 0.3606407 |
| 52_graph_feats | 0.27291408 |
| 41_graph_feats | 0.26485533 |
| 37_graph_feats | 0.12689288 |
| 30_graph_feats | 0.11398361 |
| 2_tree_feats | 0.10243105 |
| 44_graph_feats | 0.082750686 |
| 43_graph_feats | 0.0816544 |
| 53_graph_feats | 0.072278626 |
| 19_tree_feats | 0.05404014 |
| 47_graph_feats | 0.0520645 |
| 22_tree_feats | 0.051451337 |
| 36_graph_feats | 0.040479887 |
| 16_tree_feats | 0.038591255 |
| 15_tree_feats | 0.032465734 |
| 48_graph_feats | 0.020067073 |
| 40_graph_feats | 0.020005457 |
| 21_tree_feats | 0.019700417 |
| 9_tree_feats | 0.014933727 |
| 23_tree_feats | 0.012252764 |


### Train with tree features

In [77]:
model = Ridge(alpha=4.5)

In [78]:
y_train_transformed = scaler.fit_transform(y_train.reshape(-1, 1)).reshape(-1)

In [79]:
model.fit(X_train[:, :X_train.shape[1]//2], y_train_transformed)

Ridge(alpha=4.5, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [80]:
test_preds = scaler.inverse_transform(model.predict(X_test[:, :X_train.shape[1]//2]))
val_preds = scaler.inverse_transform(model.predict(X_val[:, :X_train.shape[1]//2]))

In [81]:
print("Valid RMSE =", mean_squared_error(y_val, val_preds)**0.5)
print("Valid R2-score is {0}".format(r2_score(y_val, val_preds)))

print("Test RMSE =", mean_squared_error(y_test, test_preds)**0.5)
print("Test R2-score is {0}".format(r2_score(y_test, test_preds)))

('Valid RMSE =', 1.2516059711943268)
Valid R2-score is 0.529004446022
('Test RMSE =', 1.279138606391635)
Test R2-score is 0.509250778845


In [53]:
feature_importance = model.coef_
df = pd.DataFrame(columns = ['feature importance', 'feature name'])
df['feature importance'] = feature_importance
df['feature name'] = [str(i)+ '_tree_feats' for i in range(X_train.shape[1]//2)]
df_20_most_important = df.sort_values(by='feature importance', ascending = False)[:20]

In [54]:
df_20_most_important

,feature importance,feature name
2,0.104916,2_tree_feats
19,0.052241,19_tree_feats
22,0.050642,22_tree_feats
16,0.042306,16_tree_feats
15,0.032277,15_tree_feats
21,0.023238,21_tree_feats
9,0.014161,9_tree_feats
14,0.013363,14_tree_feats
23,0.011885,23_tree_feats
13,0.007435,13_tree_feats


### Train with graph features

In [55]:
model = Ridge(alpha=1.0)

In [56]:
model.fit(X_train[:, X_train.shape[1]//2:], y_train)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [57]:
test_preds = model.predict(X_test[:, X_train.shape[1]//2:])
val_preds = model.predict(X_val[:, X_train.shape[1]//2:])

In [58]:
print("Valid RMSE =", mean_squared_error(y_val, val_preds)**0.5)
print("Valid R2-score is {0}".format(r2_score(y_val, val_preds)))

print("Test RMSE =", mean_squared_error(y_test, test_preds)**0.5)
print("Test R2-score is {0}".format(r2_score(y_test, test_preds)))

('Valid RMSE =', 1.7513535222589323)
Valid R2-score is 0.0777913424282
('Test RMSE =', 1.7450757570632138)
Test R2-score is 0.0866168912675


In [59]:
feature_importance = model.coef_
df = pd.DataFrame(columns = ['feature importance', 'feature name'])
df['feature importance'] = feature_importance
df['feature name'] = [str(i)+ '_graph_feats' for i in range(X_train.shape[1]//2, X_train.shape[1])]
df_20_most_important = df.sort_values(by='feature importance', ascending = False)[:20]

In [60]:
df_20_most_important

,feature importance,feature name
8,1.190934,36_graph_feats
2,0.993923,30_graph_feats
7,0.544352,35_graph_feats
16,0.536741,44_graph_feats
14,0.526466,42_graph_feats
25,0.367312,53_graph_feats
1,0.156772,29_graph_feats
6,0.108438,34_graph_feats
20,0.107950,48_graph_feats
19,0.041415,47_graph_feats
